In [ ]:
%load ./hello-world.ipynb

In [ ]:
import dev.langchain4j.model.chat.StreamingChatLanguageModel;
import dev.langchain4j.model.openai.OpenAiStreamingChatModel;
import dev.langchain4j.data.message.AiMessage;
import dev.langchain4j.model.output.Response;
import dev.langchain4j.model.StreamingResponseHandler;
import java.util.concurrent.BlockingQueue;

BlockingQueue<String> queue = new LinkedBlockingQueue<>();
private static final String END_SIGNAL = "END"; // 終了シグナル用

StreamingChatLanguageModel model = OpenAiStreamingChatModel.builder()
    .apiKey(API_KEY)
    .modelName(GPT_4_O_MINI)
    .logRequests(true)
    .logResponses(true)    
    .build();

String userMessage = "こんにちは";

model.generate(userMessage, new StreamingResponseHandler<AiMessage>() {

    @Override
    public void onNext(String token) {
        try {
            queue.put(token);
        } catch (InterruptedException e) {
            Thread.currentThread().interrupt();
        }
    }

    @Override
    public void onComplete(Response<AiMessage> response) {
        try {
            queue.put(END_SIGNAL);
            System.out.println("onComplete: " + response);
        } catch (InterruptedException e) {
            Thread.currentThread().interrupt();
        }
    }

    @Override
    public void onError(Throwable error) {
        error.printStackTrace();
    }
});

while (true) {
    String token = queue.poll(1, TimeUnit.SECONDS);
    if (token != null) {
        if (token.equals(END_SIGNAL)) {
            break;
        }
        System.out.println("Polled token: " + token);
    }
}